In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
nltk.download('punkt')


from functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
# if data not saved as csv, run this
import os
if not os.path.exists('data/merged_titles_labels.csv'):
    df1 = pd.read_csv('../eda/small1/labeled.csv')
    df2 = pd.read_csv('../eda/small2/labeled.csv')
    df3 = pd.read_csv('../eda/small3/labeled.csv')
    df = pd.concat([df1, df2, df3], ignore_index=True).reset_index(drop=True)
    df.to_csv('data/merged_titles_labels.csv', index=False)
    df.head()
else:
    df = pd.read_csv('data/merged_titles_labels.csv')
df.head()

,title,is_clickbait,sample
0,New organic and water-rich fertilizer will hel...,1,train
1,Watch This Guy Dynamite A Christmas Tree Just ...,0,train
2,MakeGamesWithUs (YC W12) is hiring instructors...,0,train
3,"John Carney on Dow 20,000: ‘There’s a Sort of ...",0,train
4,UK hostage Peter Moore released in Iraq,0,train


In [29]:
%run functions.py
if not os.path.exists('data/preprocessed_titles_labels.pkl'):
    df = preprocess_title(df, verbose=True)
    df.to_pickle('data/preprocessed_titles_labels.pkl') 

else:
    df = pd.read_pickle('data/preprocessed_titles_labels.pkl')
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


,title,is_clickbait,sample
0,"[new, organic, waterrich, fertilizer, help, fa...",1,train
1,"[watch, guy, dynamite, christmas, tree, time, ...",0,train
2,"[makegameswithus, yc, w12, hiring, instructor,...",0,train
3,"[john, carney, dow, twentyzero, , sort, hope,...",0,train
4,"[uk, hostage, peter, moore, released, iraq]",0,train


In [30]:
df['sample'].value_counts()

sample
train    67513
test      7503
Name: count, dtype: int64

In [31]:
df_train = df[df['sample']=='train'].reset_index(drop=True)

In [32]:
df_train['sample'].value_counts()

sample
train    67513
Name: count, dtype: int64

In [33]:
from tqdm.notebook import tqdm
import time

EPOCHS = 500
WORKERS = 16
MIN_COUNT = 1


# train many word2vec models with diferent VECTOR_SIZE and WINDOW

VECTOR_SIZEs = [
    # 100, 
    # 250, 
    # 500, 
    1000, 
    1500, 
    # 2000, 
    # 2500, 
    # 3000
]

WINDOWs = [
    3, 
    4, 
    5, 
    6, 
    7, 
    8
]
SGs = [0, 1]


##################################################
# # --uncomment for sample model training--
# EPOCHS = 200
# VECTOR_SIZEs = [500]
# WINDOWs = [4]
# SGs = [0]
##################################################


print('Start training')
# sleep 200 ms
time.sleep(0.2)

for VECTOR_SIZE in tqdm(VECTOR_SIZEs):
    print(f'Current VECTOR_SIZE: {VECTOR_SIZE}')
    for WINDOW in tqdm(WINDOWs, desc=f'WINDOW'):
        for sg in tqdm(SGs, desc=f'SG'):
            model = Word2Vec(df_train['title'], vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNT, workers=WORKERS, sg=sg)
            model.train(df_train['title'], total_examples=len(df_train['title']), epochs=EPOCHS)
            model.save(f'word2vec_models/word2vec_vs{VECTOR_SIZE}_win{WINDOW}_sg{sg}.model')

Start training


  0%|          | 0/2 [00:00<?, ?it/s]

Current VECTOR_SIZE: 1000


WINDOW:   0%|          | 0/6 [00:00<?, ?it/s]

SG:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.wv.most_similar('trump')

In [ ]:
# save vocabulary
with open('data/vocabulary.txt', 'w') as f:
    for word in model.wv.index_to_key:
        f.write(word + '\n')